# Trees

*Mô hình hoá và mô phỏng bằng Python*

Bản quyền 2021 Allen Downey

Giấy phép: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# cài đặt Pint nếu cần

try:
    import pint
except ImportError:
    !pip install pint

In [2]:
# tải về modsim.py nếu cần

from os.path import exists

filename = 'modsim.py'
if not exists(filename):
    from urllib.request import urlretrieve
    url = 'https://raw.githubusercontent.com/AllenDowney/ModSim/main/'
    local, _ = urlretrieve(url+filename, filename)
    print('Downloaded ' + local)

In [3]:
# nhập các hàm từ modsim

from modsim import *

## Mô hình hóa sự tăng trưởng của cây xanh

Nghiên cứu cụ thể này được dựa theo "[Height-Age Curves for Planted Stands of Douglas Fir, with Adjustments for Density](http://www.cfr.washington.edu/research.smc/working_papers/smc_working_paper_1.pdf)" (tạm dịch: các đường cong biểu hiện chiều cao-tuổi cây thông Douglas được gieo trồng, có điều chỉnh theo mật độ cây), một bài báo của nhóm tác giả Flewelling, Collier, Gonyea, Marshall, và Turnblom.

Bài báo cho ta các "đường cong chỉ số theo địa điểm", đây là những đường cong đồ thị biểu diên chiều cao ước tính của cây cao nhất trong một hàng cây thông Douglas như một hàm tuổi cây, đối với hàng trồng các cây cùng tuổi.

Tùy thuộc vào chất lượng của hàng ở từng địa điểm, cây có thể mọc nhanh hơn hoặc chậm hơn. Như vậy, mỗi đường đồ thị được nhận diện bởi một "chỉ số địa điểm" vốn phản ánh chất lượng của địa điểm này.

Tôi sẽ bắt đầu với vài số liệu của họ trong Bảng 1. Dưới đây là dãy các tuổi cây.

In [4]:
years = [2, 3, 4, 5, 6, 8, 10, 15, 20, 25, 30,
         35, 40, 45, 50, 55, 60, 65, 70]

Và đây là một dãy các chiều cao cây ở địa điểm có chỉ số 45, điều này cho thấy chiều cao ở tuổi cây 30 năm là 45 feet.

In [5]:
site45 = TimeSeries([1.4, 1.49, 1.75, 2.18, 2.78, 4.45, 6.74,
                    14.86, 25.39, 35.60, 45.00, 53.65, 61.60,
                    68.92, 75.66, 81.85, 87.56, 92.8, 97.63],
                    index=years)

Đây là dãy số liệu ở địa điểm có chỉ số 65.

In [6]:
site65 = TimeSeries([1.4, 1.56, 2.01, 2.76, 3.79, 6.64, 10.44, 
                    23.26, 37.65, 51.66, 65.00, 77.50, 89.07, 
                    99.66, 109.28, 117.96, 125.74, 132.68, 138.84],
                    index=years)

Và ở địa điểm có chỉ số 85.

In [7]:
site85 = TimeSeries([1.4, 1.8, 2.71, 4.09, 5.92, 10.73, 16.81, 
                 34.03, 51.26, 68.54, 85, 100.34, 114.33,
                 126.91, 138.06, 147.86, 156.39, 163.76, 170.10],
               index=years)

Các đường cong sẽ có hình dạng như dưới đây:

In [8]:
site85.plot(label='SI 85')
site65.plot(label='SI 65')
site45.plot(label='SI 45')
decorate(xlabel='Time (years)',
         ylabel='Height (feet)')

Tôi chỉ làm ví dụ với dữ liệu chỉ số (SI) là 65. Bạn có thể tập chạy lại sổ tính notebook này cho các đường cong khác.

In [9]:
data = site65

## Mô hình 1

Để bắt đầu, ta hãy giả sử rằng khả năng cây trồng tăng trọng thì bị hạn chế bởi diện tích nó phơi ra ngoài ánh mặt trời, còn tốc độ tăng trưởng (về khối lượng) thì tỉ lệ thuận với diện tích đó. Như vậy, ta có thể viết:

$$ m_{n+1} = m_n + \alpha A$$

trong đó $m_n$ là khối lượng cây tại bước thời gian $n$, $A$ là diện tích phơi nắng, còn $\alpha$ là một hệ số sinh trưởng mà ta chưa biết.

Để đi từ $m$ đến $A$, tôi sẽ giả sử thêm rằng khối lượng thì tỉ lệ với chiều cao lũy thừa một số mũ chưa biết khác:

$$ m = \beta h^D $$

trong đó $h$ là chiều cao, $eta$ là một hằng số tỉ lệ chưa biết, và $D$ là đại lượng liên hệ giữa chiều cao và khối lượng. Để bắt đầu, tôi sẽ giả sử $D=3$, và sẽ còn kiểm tra lại giả sử này sau.

Cuối cùng, tôi sẽ giả sử rằng diện tích thì tỉ lệ với bình phương của chiều cao:

$$ A = \gamma h^2$$

Tôi tính chiều cao theo feet, và chọn các đơn vị cho khối lượng và diện tích sao cho $\beta=1$ và $\gamma=1$.

Tổng hợp mọi thứ lại, ta có thể viết được một phương trình sai phân cho chiều cao:

$$ h_{n+1}^D = h_n^D + \alpha h_n^2 $$

Bây giờ ta hãy mô phỏng hệ thống này. Sau đây là một đối tượng hệ thống cùng các tham số và điều kiện đầu.

In [10]:
alpha = 7
dim = 3

t_0 = data.index[0]
h_0 = data[t_0]
t_end = data.index[-1]

system = System(alpha=alpha, 
                dim=dim, 
                h_0=h_0, 
                t_0=t_0, 
                t_end=t_end)

Và sau đây là một hàm cập nhật. Hàm này nhận vào tham số là chiều cao hiện thời và trả lại chiều cao tại bước thời gian kế tiếp.

In [11]:
def update(height, t, system):
    """Cập nhật chiều cao dựa theo mô hình cấp số nhân.
    
    height: chiều cao hiện thời tính bằng feet
    t: năm tính toán
    system: đối tượng hệ thống cùng với tham số mô hình
    """
    area = height**2
    mass = height**system.dim
    mass += system.alpha * area
    return mass**(1/system.dim)

Tôi sẽ thử hàm cập nhật với các điều kiện ban đầu.

In [12]:
update(h_0, t_0, system)

Đây là phiên bản `run_simulation` thông thường của ta.

In [13]:
def run_simulation(system, update_func):
    """Mộ phỏng hệ thống bằng một hàm cập nhật bất kì.
    
    system: đối tượng hệ thống System
    update_func: hàm cập nhật, để tính số cá thể trong năm tới
    
    trả lại: TimeSeries
    """
    results = TimeSeries()
    results[system.t_0] = system.h_0
    
    for t in linrange(system.t_0, system.t_end-1):
        results[t+1] = update_func(results[t], t, system)
        
    return results

Và sau đây là cách ta chạy nó.

In [14]:
results = run_simulation(system, update)
results.tail()

Và kết quả sẽ trông như sau:

In [15]:
def plot_results(results, data):
    results.plot(style=':', label='model', color='gray')
    data.plot(label='data')
    decorate(xlabel='Time (years)',
             ylabel='Height (feet)')
    
plot_results(results, data)

Mô hình đã hội tụ về một đường thẳng.

Tôi đã chọn giá trị của `alpha` sao cho càng khớp điểm số liệu bao nhiêu thì tốt bấy nhiêu, nhưng rõ ràng ở số liệu có độ cong mà mô hình không bắt được.

Đây là những sai số, tức là khác biệt giữa mô hình cây và dữ liệu.

In [16]:
errors = results - data
errors.dropna()

Và đây là sai số tuyệt đối trung bình.

In [17]:
def mean_abs_error(results, data):
    return (results-data).abs().mean()

mean_abs_error(results, data)

Mô hình này có thể giải thích tại sao chiều cao của cây tăng gần như là tuyến tính:

1. Nếu diện tích tỉ lệ với $h^2$ còn khối lượng thì tỉ lệ với $h^3$, và

2. Mức thay đổi về khối lượng thì tỉ lệ với diện tích, và

3. Chiều cao tăng trưởng một cách tuyến tính, thì khi đó:

4. Diện tích tăng tỉ lệ với $h^2$, và

5. Khối lượng tăng tỉ lệ với $h^3$.

Nếu mục đích là chỉ nhằm giải thích (xấp xỉ) sự tăng trưởng tuyến tính, thì ta có thể dừng lại ở đây. Song mô hình vẫn chưa khớp với dữ liệu lắm, và mô hình hàm ý rằng cây vẫn cứ tiếp tục tăng trưởng mãi.

Vì vậy ta cần phải làm tốt hơn nữa.

## Model 2

As a second attempt, let's suppose that we don't know $D$.  In fact, we don't, because trees are not like simple solids; they are more like fractals, which have [fractal dimension](https://en.wikipedia.org/wiki/Fractal_dimension).

I would expect the fractal dimension of a tree to be between 2 and 3, so I'll guess 2.5.

In [18]:
alpha = 7
dim = 2.5

I'll wrap the code from the previous section is a function that takes the parameters as inputs and makes a `System` object.

In [19]:
def make_system(params, data):
    """Makes a System object.
    
    params: sequence of alpha, dim
    data: Series
    
    returns: System object
    """
    alpha, dim = params
    
    t_0 = data.index[0]
    t_end = data.index[-1]
    h_0 = data[t_0]

    return System(alpha=alpha, dim=dim, 
                  h_0=h_0, t_0=t_0, t_end=t_end)

Here's how we use it.

In [20]:
params = alpha, dim
system = make_system(params, data)

With different values for the parameters, we get curves with different behavior.  Here are a few that I chose by hand.

In [21]:
def run_and_plot(alpha, dim, data):
    params = alpha, dim
    system = make_system(params, data)
    results = run_simulation(system, update)
    results.plot(style=':', color='gray', label='_nolegend')

In [22]:
run_and_plot(0.145, 2, data)
run_and_plot(0.58, 2.4, data)
run_and_plot(2.8, 2.8, data)
run_and_plot(6.6, 3, data)
run_and_plot(15.5, 3.2, data)
run_and_plot(38, 3.4, data)

data.plot(label='data')
decorate(xlabel='Time (years)',
             ylabel='Height (feet)')

To find the parameters that best fit the data, I'll use `leastsq`.

We need an error function that takes parameters and returns errors:

In [23]:
def error_func(params, data, update_func):
    """Runs the model and returns errors.
    
    params: sequence of alpha, dim
    data: Series
    update_func: function object
    
    returns: Series of errors
    """
    print(params)
    system = make_system(params, data)
    results = run_simulation(system, update_func)
    return (results - data).dropna()

Here's how we use it:

In [24]:
errors = error_func(params, data, update)

Now we can pass `error_func` to `leastsq`, which finds the parameters that minimize the squares of the errors.

In [25]:
best_params, details = leastsq(error_func, params, data, update)

In [26]:
print(details.success)

Using the best parameters we found, we can run the model and plot the results.

In [27]:
system = make_system(best_params, data)
results = run_simulation(system, update)
plot_results(results, data)

The mean absolute error is better than for Model 1, but that doesn't mean much.  The model still doesn't fit the data well.

In [28]:
mean_abs_error(results, data)

And the estimated fractal dimension is 3.11, which doesn't seem likely.

Let's try one more thing.

## Model 3

Models 1 and 2 imply that trees can grow forever, but we know that's not true.  As trees get taller, it gets harder for them to move water and nutrients against the force of gravity, and their growth slows.

We can model this effect by adding a term to the model similar to what we saw in the logistic model of population growth.  Instead of assuming:

$ m_{n+1} = m_n + \alpha A $ 

Let's assume

$ m_{n+1} = m_n + \alpha A (1 - h / K) $

where $K$ is similar to the carrying capacity of the logistic model.  As $h$ approaches $K$, the factor $(1 - h/K)$ goes to 0, causing growth to level off.

Here's what the implementation of this model looks like:

In [30]:
alpha = 2.0
dim = 2.5
K = 150

params = [alpha, dim, K]

Here's an updated version of `make_system`

In [31]:
def make_system(params, data):
    """Makes a System object.
    
    params: sequence of alpha, dim, K
    data: Series
    
    returns: System object
    """
    alpha, dim, K = params
    
    t_0 = data.index[0]
    t_end = data.index[-1]
    h_0 = data[t_0]

    return System(alpha=alpha, dim=dim, K=K, 
                  h_0=h_0, t_0=t_0, t_end=t_end)

Here's the new `System` object.

In [32]:
system = make_system(params, data)

And here's the new update function.

In [33]:
def update3(height, t, system):
    """Update height based on geometric model with growth limiting term.
    
    height: current height in feet
    t: what year it is
    system: system object with model parameters
    """
    area = height**2
    mass = height**system.dim
    mass += system.alpha * area * (1 - height/system.K)
    return mass**(1/system.dim)

As always, we'll test the update function with the initial conditions.

In [34]:
update3(h_0, t_0, system)

And we'll test the error function with the new update function.

In [35]:
error_func(params, data, update3)

Now let's search for the best parameters.

In [36]:
best_params, details = leastsq(error_func, params, data, update3)

In [37]:
details.success

With these parameters, we can fit the data much better.

In [38]:
system = make_system(best_params, data)
results = run_simulation(system, update3)
plot_results(results, data)

And the mean absolute error is substantially smaller.

In [39]:
mean_abs_error(results, data)

The estimated fractal dimension is about 2.6, which is plausible; it suggests that if you double the height of the tree, the mass grows by a factor of $2^{2.6}$

In [40]:
2**2.6

In other words, the mass of the tree scales faster than area, but not as fast as it would for a solid 3-D object.

What is this model good for?

1) It offers a possible explanation for the shape of tree growth curves.

2) It provides a way to estimate the fractal dimension of a tree based on a growth curve (probably with different values for different species).

3) It might provide a way to predict future growth of a tree, based on measurements of past growth.  As with the logistic population model, this would probably only work if we have observed the part of the curve where the growth rate starts to decline.

## Analysis

With some help from my colleague, John Geddes, we can do some analysis.

Starting with the difference equation in terms of mass:
 
$m_{n+1} = m_n + \alpha A (1 - h / K) $

We can write the corresponding differential equation:

(1) $ \frac{dm}{dt} = \alpha A (1 - h / K) $

With

(2) $A = h^2$

and

(3) $m = h^D$

Taking the derivative of the last equation yields

(4) $\frac{dm}{dt} = D h^{D-1} \frac{dh}{dt}$

Combining (1), (2), and (4), we can write a differential equation for $h$:

(5) $\frac{dh}{dt} = \frac{\alpha}{D} h^{3-D} (1 - h/K)$

Now let's consider two cases:

* With infinite $K$, the factor $(1 - h/K)$ approaches 1, so we have Model 2.

* With finite $K$, we have Model 3.

### Model 2

Within Model 2, we'll consider two special cases, with $D=2$ and $D=3$.

With $D=2$, we have

$\frac{dh}{dt} = \frac{\alpha}{2} h$

which yields exponential growth with parameter $\alpha/2$.

With $D=3$, we have Model 1, with this equation:

$\frac{dh}{dt} = \frac{\alpha}{3}$

which yields linear growth with parameter $\alpha/3$. This result explains why Model 1 is linear.

### Model 3

Within Model 3, we'll consider two special cases, with $D=2$ and $D=3$.

With $D=2$, we have

$\frac{dh}{dt} = \frac{\alpha}{2} h (1 - h/K)$

which yields logisitic growth with parameters $r = \alpha/2$ and $K$.

With $D=3$, we have

$\frac{dh}{dt} = \frac{\alpha}{3} (1 - h/K)$

which yields a first order step response; that is, it converges to $K$ like a negative exponential:

$ h(t) = c \exp(-\frac{\alpha}{3K} t) + K $

where $c$ is a constant that depends on the initial conditions.

**Open Exercise** Find an analytic solution when $D$ is between 2 and 3, and compare it to the data.  Note: The parameters we estimated for the difference equation might not be right for the differential equation.

Additional resources:

Garcia, [A stochastic differential equation model for the
height growth of forest stands](http://citeseerx.ist.psu.edu/viewdoc/download;jsessionid=664FED1E46ABCBF6E16741C294B79976?doi=10.1.1.608.81&rep=rep1&type=pdf)

[EasySDE software and data](http://forestgrowth.unbc.ca/)